In [1]:
import tensorflow as tf

fashion=tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test)=fashion.load_data()

print (x_train.shape)
print (y_train.shape)

print (x_test.shape)
print (y_test.shape)

In [16]:
len(set(y_train))

10

In [8]:
print(x_test[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   3   1   0   0   7   0  37   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   1   2   

In [39]:
BATCH_SIZE=128
EPOCH=50
cls_num=len(set(y_train))
image_shape=(28,28)

In [40]:
def create_model(features,feature_columns,output_cls):
    
    input_layer=tf.feature_column.input_layer(features,feature_columns)
    
    net = tf.reshape(input_layer, [-1, 28, 28, 1]) 
    
#     tf.layers.conv2d   isdeprecated ,keras.layers.conv2d
    conv1=tf.keras.layers.Conv2D(filters=16,kernel_size=[3,3],activation='relu',name='conv1')(net)
    pool1=tf.keras.layers.MaxPool2D(pool_size=(3,3))(conv1)
    conv2=tf.keras.layers.Conv2D(filters=8,kernel_size=[5,5],activation='relu',name='conv2')(pool1)
    flat=tf.keras.layers.Flatten()(conv2)
    dens1=tf.keras.layers.Dense(units=128,activation='relu',name='dense1')(flat)
    logits=tf.keras.layers.Dense(units=output_cls,name='dense_output')(dens1)
    return logits
def  model_fn_builder(lr):
    # 该方法实际 创建 estimator的model_fn
    # 可以 有其他操作
    def model_fn(features, labels, mode, params,config): #estimator需要的model_fn 参数固定
        '''
        features: from input_fn的返回  切记返回的顺序
        labels： from input_fn 的返回  切记返回的顺序
        mode: tf.estimator.ModeKeys实例的一种
        params: 在初始化estimator时 传入的参数列表，dict形式,或者直接使用self.params也可以
        config:初始化estimator时 的 Runconfig

        '''
        logits=create_model(features,params['feature_columns'],params['output_cls'])
        
        pre_cls=tf.math.argmax(input=logits,axis=1)
        pre_prob=tf.nn.softmax(logits=logits,axis=1)
        
        is_predict=mode==tf.estimator.ModeKeys.PREDICT
        if not is_predict:
            # train .eval
            loss=tf.losses.sparse_softmax_cross_entropy(labels=tf.cast(labels,tf.int32),logits=logits)

            def metric_fn(labels,predictions):
                '''
                define metrics
                '''
                accuracy,accuracy_update=tf.metrics.accuracy(labels=labels,predictions=predictions,name='iris_accuracy')
    #                 auc=tf.metrics.auc(labels=labels,predictions=predictions,name='iris_auc')
                recall,recall_update=tf.metrics.recall(labels=labels,predictions=predictions,name='iris_recall')
                precision,precision_update=tf.metrics.precision(labels=labels,predictions=predictions,name='iris_precision')
    #                 with tf.control_dependencies([recall,precision]):
    #                     f1_score=f1(recall=recall,precision=precision)

                return {
                    'accuracy':(accuracy,accuracy_update),
                    'recall':(recall,recall_update),
                    'precision':(precision,precision_update)                  
                }


            if mode==tf.estimator.ModeKeys.EVAL:
                return tf.estimator.EstimatorSpec(mode=mode,loss=loss,eval_metric_ops=metric_fn(labels,pre_cls))

            # train process
            train_op=tf.train.AdamOptimizer(learning_rate=lr).minimize(loss=loss,global_step=tf.train.get_global_step())
            return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op,eval_metric_ops=metric_fn(labels,pre_cls))


        else:
            predictions={'predict_cls':pre_cls,'predict_pro':pre_prob}
            return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions)     
    return model_fn
      

In [41]:
def input_fn_builder(x,y,batch_size,epochs,is_train=True):
    '''
    创建 输入函数闭包
    '''
    # 可以执行其它操作
    
    def input_fn():
        dataset=tf.data.Dataset.from_tensor_slices(({'images':x_train},y_train) )  
        if is_train:
            dataset=dataset.shuffle(1000).repeat(epochs)
        dataset=dataset.batch(batch_size)
        return dataset # 返回的 顺序要和 model_fn一致 或者 dataset元素 格式为（features,label）元组 也可以
    return input_fn

In [42]:
model_dir=r'F:\testDemo\AI\estimator\model\fashion'
params={}
feature_columns=[tf.feature_column.numeric_column('images',shape=image_shape)]

output_cls=cls_num
params['feature_columns']=feature_columns
params['output_cls']=output_cls


In [43]:
feature_columns

[NumericColumn(key='images', shape=(28, 28), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [44]:
config=tf.estimator.RunConfig(save_checkpoints_steps=100)

estimator=tf.estimator.Estimator(model_fn=model_fn_builder(0.001),model_dir=model_dir,params=params,config=config)

INFO:tensorflow:Using config: {'_model_dir': 'F:\\testDemo\\AI\\estimator\\model\\fashion', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001B9B0211FD0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [45]:
train=estimator.train(input_fn=input_fn_builder(x=x_train,y=y_train,batch_size=BATCH_SIZE,epochs=EPOCH,is_train=True),steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from F:\testDemo\AI\estimator\model\fashion\model.ckpt-938
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 938 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:loss = 0.50735295, step = 939
INFO:tensorflow:Saving checkpoints for 1038 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 27.8764
INFO:tensorflow:loss = 0.42255113, step = 1039 (3.589 sec)
INFO:tensorflow:Saving checkpoints for 1138 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 30.73
INFO:tensorflow:loss = 0.47224864, step = 1139 (3.254 sec)
INFO:tensorflow:Saving checkpoints for 1238 into F:\te

INFO:tensorflow:global_step/sec: 27.3896
INFO:tensorflow:loss = 0.37277585, step = 4739 (3.649 sec)
INFO:tensorflow:Saving checkpoints for 4838 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 23.6595
INFO:tensorflow:loss = 0.37412894, step = 4839 (4.227 sec)
INFO:tensorflow:Saving checkpoints for 4938 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 28.7714
INFO:tensorflow:loss = 0.288697, step = 4939 (3.478 sec)
INFO:tensorflow:Saving checkpoints for 5038 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 29.0183
INFO:tensorflow:loss = 0.2763981, step = 5039 (3.444 sec)
INFO:tensorflow:Saving checkpoints for 5138 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 22.8926
INFO:tensorflow:loss = 0.31769186, step = 5139 (4.367 sec)
INFO:tensorflow:Saving checkpoints for 5238 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INF

INFO:tensorflow:loss = 0.26559156, step = 8839 (3.150 sec)
INFO:tensorflow:Saving checkpoints for 8938 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 31.1782
INFO:tensorflow:loss = 0.2205996, step = 8939 (3.207 sec)
INFO:tensorflow:Saving checkpoints for 9038 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 31.4427
INFO:tensorflow:loss = 0.23598878, step = 9039 (3.179 sec)
INFO:tensorflow:Saving checkpoints for 9138 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 31.1181
INFO:tensorflow:loss = 0.33628604, step = 9139 (3.216 sec)
INFO:tensorflow:Saving checkpoints for 9238 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 28.9266
INFO:tensorflow:loss = 0.21565297, step = 9239 (3.456 sec)
INFO:tensorflow:Saving checkpoints for 9338 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 30.5995
I

In [46]:
estimator.evaluate(input_fn=input_fn_builder(x=x_test,y=y_test,batch_size=BATCH_SIZE,epochs=EPOCH,is_train=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-03T10:17:10Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from F:\testDemo\AI\estimator\model\fashion\model.ckpt-10938
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-03-10:17:16
INFO:tensorflow:Saving dict for global step 10938: accuracy = 0.91565, global_step = 10938, loss = 0.22562842, precision = 0.9838451, recall = 0.98907405
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10938: F:\testDemo\AI\estimator\model\fashion\model.ckpt-10938


{'accuracy': 0.91565,
 'loss': 0.22562842,
 'precision': 0.9838451,
 'recall': 0.98907405,
 'global_step': 10938}